In [2]:
from pyscf import gto, scf, dft
from pyscf.lo import orth


import numpy as np
from IPython.display import clear_output
import pandas as pd
from pathlib import Path


In [3]:
def np_to_scfinput(atoms, coords):
    '''
    Input: tuple of (coordinates: np.array; atom representation: tuple of strings)
    '''
    # safeguarding here
    # convertion
    raw = []
    for atom, coord in zip(atoms, coords):
        raw.append(' '.join([atom, *coord.astype(str)]))
    
    return ' ;'.join(raw)

def add_gaussian_noise(coords, sigma=0.02, seed=None):
    '''
    Input: np array shape (N,3)
    First row is central atom: 0,0,0. Or any other atom that is kept at the origin
    Second and third rows are hydrogens / all other atoms
    '''
    # default_rng is the new np random Generator. good to use!
    rng = np.random.default_rng(seed)
    noise = rng.normal(loc=0.0, scale=sigma, size=coords.shape)
    coords_w_noise = coords + noise

    # force the first row (central atom) to be 0,0,0
    coords_w_noise[0] = [0,0,0]
    return coords_w_noise

def scf_features(mf, mol):

    # Grab SAAO coefficients from a molecule
    S = mf.get_ovlp()                               # AO overlap (nao×nao)
    C_sao = orth.orth_ao(mol, method='lowdin', s=S)     # Symmetric orthogonalizer: for transformation to SAAO basis

    # Obtain the AO basis matrices:
    H_ao  = mf.get_hcore()                              # Core Hamiltonian in AO basis
    F_ao  = mf.get_fock(dm=mf.make_rdm1())              # Fock in AO basis
    P_ao  = mf.make_rdm1()                              # Density in AO basis
    J_ao, K_ao = mf.get_jk(mf, dm=mf.make_rdm1())

    # Project into SAAO basis:
    H_sao = C_sao.T @ H_ao @ C_sao
    F_sao = C_sao.T @ F_ao @ C_sao
    P_sao = C_sao.T @ P_ao @ C_sao
    J_sao = C_sao.T @ J_ao @ C_sao
    K_sao = C_sao.T @ K_ao @ C_sao

    # diagonal blocks per atom:
    diag_blocks = []
    atom_slices = mol.aoslice_by_atom()
    for (_, _, ist, sto) in atom_slices:
        diag_blocks.append(np.stack([
        np.diag(F_sao[ist:sto, ist:sto]),
        np.diag(J_sao[ist:sto, ist:sto]),
        np.diag(K_sao[ist:sto, ist:sto]),
        np.diag(P_sao[ist:sto, ist:sto]),
        np.diag(H_sao[ist:sto, ist:sto]),
        ], axis=1))

    # Pad each block to (max_n,5)
    max_n = max(b.shape[0] for b in diag_blocks)
    padded = []
    for b in diag_blocks:
        pad = np.zeros((max_n - b.shape[0], 5), dtype=b.dtype)
        padded.append(np.vstack([b, pad]))

    return np.stack(padded, axis=0)  # shape (natoms, max_n, 5)


## Prepare water molecule file (default)

1. Water molecule

In [4]:
# actual water molecule data
bond_len = 0.9572
bond_ang = 104.5
atoms = ('O','H','H') # what atom that each row corresponds to 

2. $CO_2$ molecule

In [11]:
bond_len = 1.163
bond_ang = 180
atoms = ('C','O','O') # what atom that each row corresponds to 


In [5]:
x = bond_len * np.sin(np.radians(bond_ang/2))
y = bond_len * np.cos(np.radians(bond_ang/2))

coords = np.array([[0,0,0],[x, y, 0],[-x,y,0]]) # put in the actual data
print(coords)

[[ 0.          0.          0.        ]
 [ 0.75684806  0.58601438  0.        ]
 [-0.75684806  0.58601438  0.        ]]


3. More complex CH3OH molecule: 

In [10]:

atoms = ('C', 'H', 'H', 'H', 'O', 'H')

coords = np.array([
    [ 0.0000,   0.0000,   0.0000],   # C
    [ 0.4970,   0.0280,   0.9736],   # H
    [ 0.2846,   0.8860,  -0.5809],   # H
    [ 0.3124,  -0.9001,  -0.5437],   # H
    [-1.3613,  -0.0150,   0.3035],   # O
    [-1.8357,  -0.0394,  -0.5177],   # H
])

4. Tryptophan

In [ ]:
['N', 'C', 'C', 'O', 'O', 'C', 'H', 'H', 'C', 'C', 'N', 'H', 
 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H', 
 'H', 'H', 'H']

np.array([
    [-1.4575,  0.0290,  0.2665],
    [ 0.0000,  0.0000,  0.0000],  # central C at origin
    [ 0.2049, -0.1613, -1.5007],
    [-0.1837, -1.0480, -2.2388],
    [ 0.9382,  0.8140, -2.0847],
    [ 0.7620, -1.1241,  0.7262],
    [ 1.8248, -1.1086,  0.4135],
    [ 0.3689, -2.1185,  0.4290],
    [ 0.6792, -0.9733,  2.1938],
    [ 1.4516, -0.1389,  2.9777],
    [ 1.0281, -0.2452,  4.3205],
    [ 1.5808,  0.0301,  5.0911],
    [ 0.0408, -1.2548,  4.3923],
    [-0.2183, -1.6919,  3.0664],
    [-1.2026, -2.6616,  2.8367],
    [-1.4098, -3.0047,  1.8173],
    [-1.8909, -3.1668,  3.9209],
    [-2.6632, -3.9278,  3.7677],
    [-1.6234, -2.7267,  5.2321],
    [-2.1946, -3.1583,  6.0609],
    [-0.6623, -1.7705,  5.4899],
    [-0.4535, -1.4284,  6.5080],
    [ 2.2433,  0.5433,  2.6641],
    [-1.6012,  0.1888,  1.2420],
    [-1.8690, -0.8458,  0.0132],
    [ 0.4013,  0.9898,  0.3360],
    [ 1.0075,  0.6539, -3.0209]
    ], dtype=float32
)

The code

In [ ]:

#####################
data_dir = Path('./h2o_energies_noise_scf_features') # change this
file_name = 'h2o_noise_scf_dft_b3lyp_def2-SVP'
# mf_scanner = gto.M().apply(dft.RKS).set(xc='b3lyp', basis='def2-SVP').as_scanner() # an easy way to ask for energy only
####################

data_dir.mkdir(exist_ok=True)

sigma = 0.02

for i in range(875, 1000):
    coords_w_noise = add_gaussian_noise(coords, sigma=sigma, seed=i)
    inp = np_to_scfinput(atoms, coords_w_noise)
    mol = gto.M(atom=inp, basis='def2-SVP', charge=0, spin=0)
    rhf = scf.RHF(mol)
    e = rhf.kernel()
    features = scf_features(rhf, mol)


    np.savez_compressed(
        data_dir / f'{file_name}_{i}.npz', # change y
        coords=coords_w_noise,
        energy = e,
        noise_sigma = sigma,
        features = features
    )

converged SCF energy = -75.9603498635005
converged SCF energy = -75.9610651630513
converged SCF energy = -75.960017806597
converged SCF energy = -75.9604172268138
converged SCF energy = -75.9600793777882
converged SCF energy = -75.9584196569018
converged SCF energy = -75.9607542556728
converged SCF energy = -75.9589121727985
converged SCF energy = -75.9598187117938
converged SCF energy = -75.9605211033097
converged SCF energy = -75.9605478596525
converged SCF energy = -75.9608821222584
converged SCF energy = -75.9586345158646
converged SCF energy = -75.9584033593258
converged SCF energy = -75.9578076701854
converged SCF energy = -75.9602809820355
converged SCF energy = -75.9601558425837
converged SCF energy = -75.9596913156088
converged SCF energy = -75.9599817793515
converged SCF energy = -75.9609377040524
converged SCF energy = -75.9604096323175
converged SCF energy = -75.9540073872269
converged SCF energy = -75.960986809603
converged SCF energy = -75.9611411943316
converged SCF ener

In [ ]:
# inspection of one single file
index = 6
file_path = data_dir / f'{file_name}_{index}.npz'
data = np.load(file_path)
print(
    data['coords'], data['energy'], data['noise_sigma'], 
      # data['features'].shape
)

[[ 0.          0.          0.        ]
 [ 0.92395162 -0.21938935  0.02704284]
 [-0.91363515 -0.20972138  0.00579915]] -75.92283626317517 0.02


In [18]:
# Evaluation on: CNN, GNN. We have x_test, y_test
from numpy.random import default_rng

def translation(X, offset=10.0):
    '''
    Offset: a (3,) vector (tuple) or a scaler. 
    '''
    off = np.asarray(offset)
    if off.ndim == 0:
        off = np.array([off, off, off])
    return X + off  # broadcasting: (N,n_atoms,3) + (3,) → (N,n_atoms,3)

def rotate_180_y_axis(X):
    R = np.array([[-1, 0, 0],
              [ 0, 1, 0],
              [ 0, 0,-1]])
    return X @ R.T

def permutation(X, seed=None):
    N, n_atoms, xyz = X.shape
    rng = default_rng(seed)          # create a Generator with seed 42
    perm = rng.permutation(n_atoms)
    return X[:, perm, :]

In [19]:
# for test dataset, apply transformation and save again
c_orig = Path('./test_xyz_co2.npy')
c_orig = np.load(c_orig)

X = []
C = []
for coords in c_orig:
    # new_coords = translation(coords, offset=100.0)
    new_coords = rotate_180_y_axis(coords)
    # new_coords = permutation(new_coords, seed=42)  # random permutation

    inp = np_to_scfinput(atoms, new_coords)
    mol = gto.M(atom=inp, basis='def2-SVP', charge=0, spin=0)
    rhf = scf.RHF(mol)
    e = rhf.kernel()

    X.append(scf_features(rhf, mol)) # shape: (N, 3, 14, 5)
    C.append(new_coords)


N, n_atoms, n_features, channels = np.stack(X).shape
X = np.stack(X).reshape(N, n_atoms, n_features * channels)  # (N, n_atoms, n_features * channels    )
print(X.shape)  # (N, n_atoms, n_features * channels)

np.save('./test_scf_co2_rotated.npy', np.stack(X))
np.save('./test_xyz_co2_rotated.npy', np.stack(C))

converged SCF energy = -187.487340705686
converged SCF energy = -187.478203658201
converged SCF energy = -187.484858776485
converged SCF energy = -187.485782327445
converged SCF energy = -187.480180131439
converged SCF energy = -187.484907034054
converged SCF energy = -187.476636176373
converged SCF energy = -187.483817971318
converged SCF energy = -187.48114502558
converged SCF energy = -187.482162303084
converged SCF energy = -187.482174325598
converged SCF energy = -187.47549690296
converged SCF energy = -187.47993274454
converged SCF energy = -187.480220809503
converged SCF energy = -187.484125876471
converged SCF energy = -187.483650562635
converged SCF energy = -187.480955946582
converged SCF energy = -187.48426609422
converged SCF energy = -187.482139997579
converged SCF energy = -187.485841205418
converged SCF energy = -187.471798238433
converged SCF energy = -187.477395940096
converged SCF energy = -187.481163604834
converged SCF energy = -187.48616424847
converged SCF energy 